# Geographic Optimization

## 1. Distance Calculation

Distances between each of the 16 focus compounds and all approx. 8000 zip codes in Germany.

### Imports

In [6]:
import pandas as pd
import numpy as np
import pgeocode
import haversine as hs

### Data Preparation

#### Create Dataframes

In [7]:
zipcodes_df = pd.read_csv('zipcodes.csv',usecols=['zipcode'],dtype='str')
zipcodes_df

,zipcode
0,01067
1,01069
2,01097
3,01099
4,01108
...,...
8169,99988
8170,99991
8171,99994
8172,99996


In [8]:
compounds_df = pd.read_csv('compounds_addresses.csv')
compounds_df

,compound_name,compound_address
0,AKB Kitzingen,"AKB Compound Kitzingen, Larson Barracks 53, 97..."
1,AKB Dortmund,"AKB Compound Dortmund, Dammstraße 25, 44145 Do..."
2,AKB Zörbig,"AKB Compound Zörbig, Jeßnitzer Str. 26, 06780 ..."
3,AKB Schöneck,"AKB Compound Schöneck, Windecker Str. 2, 61137..."
4,AKB Buch,"AKB Compound Buch, An der Lehmgrube 1, 89290 Buch"
5,Mosolf Etzin,ACM Auto-Service und Umschlag-Center Mosolf Et...
6,Mosolf Kippenheim,"Mosolf Compound, Freimatte 25, 77971 Kippenheim"
7,BLG Kelheim,"Hafenstraße 33, 93342 Saal an der Donau"
8,BLG Duisburg,"BLG AutoTerminal Duisburg GmbH & Co. KG, Rotte..."
9,BLG Neuss,"ATN Autoterminal Neuss, Floßhafenstr. 30, 4146..."


#### Convert zip code to longitude and latitude

In [9]:
nomi = pgeocode.Nominatim('de')

In [10]:
for index,row in zipcodes_df.iterrows():
    query = nomi.query_postal_code(zipcodes_df.iat[index,0])
    zipcodes_df.at[index,'lat']= query['latitude']
    zipcodes_df.at[index,'long']= query['longitude']

In [11]:
compounds_df['zipcode'] = compounds_df['compound_address'].str.findall(r'([0-9]\d+)').apply(lambda x: x[-1] if len(x) >= 1 else '')


In [12]:
for index,row in compounds_df.iterrows():
    query = nomi.query_postal_code(compounds_df.iat[index,2])
    compounds_df.at[index,'lat']= query['latitude']
    compounds_df.at[index,'long']= query['longitude']

#### Add coordinate column (necessary for usage of Haversine) 

In [13]:
zipcodes_df['coor']=list(zip(zipcodes_df.lat,zipcodes_df.long))
compounds_df['coor']=list(zip(compounds_df.lat,compounds_df.long))

In [14]:
zipcodes_df

,zipcode,lat,long,coor
0,01067,51.054700,13.726900,"(51.0547, 13.7269)"
1,01069,51.043000,13.737300,"(51.043, 13.7373)"
2,01097,51.071400,13.739900,"(51.0714, 13.7399)"
3,01099,51.078300,13.805100,"(51.0783, 13.8051)"
4,01108,51.155733,13.782467,"(51.15573333333333, 13.782466666666666)"
...,...,...,...,...
8169,99988,51.172900,10.290450,"(51.1729, 10.29045)"
8170,99991,51.148467,10.553300,"(51.14846666666667, 10.5533)"
8171,99994,51.239850,10.670850,"(51.23985, 10.67085)"
8172,99996,51.288800,10.580350,"(51.2888, 10.58035)"


### Calculate Distances

In [ ]:
def distance_from(loc1,loc2):
    '''This function defines the distance between customers (loc1) and compound (loc2)'''
    dist = hs.haversine(loc1,loc2)
    return round(dist,2)

In [ ]:
full_distances_df = zipcodes_df.copy()

In [ ]:
for _,row in compounds_df.iterrows():
    full_distances_df[row.compound_name]=full_distances_df['coor'].apply(lambda x: distance_from(row['coor'],x))

In [ ]:
distances = full_distances_df.drop(columns=['lat','long','coor'],axis=1)

In [ ]:
distances.set_index('zipcode', inplace=True)

In [ ]:
distances

Result:
For every zipcode, the distances (in km) to every compound are given. 
As it's stored in a pandas Dataframe, further investigations can be easily done (p.eg. seeing the minimum per row etc.).

### Calculate Driving Distance

In [ ]:
import requests
import json
from tqdm import tqdm

In [ ]:
def request_driving_distance_in_meters_from_api(loc1,loc2):
    '''Requests from OpenStreetMap to calculate Driving Distance between customer and compound'''
    r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{loc1[1]},{loc1[0]};{loc2[1]},{loc2[0]}?overview=false""")
    content = json.loads(r.content)
    if 'routes' in content:
        route_1 = content['routes'][0]
        return route_1['distance']
    else:
        return 0.0

In [ ]:
tqdm.pandas()
driving_distances_df = zipcodes_df.copy()
for _,row in compounds_df.iterrows():
    driving_distances_df[row.compound_name]=driving_distances_df['coor'].progress_apply(lambda x: request_driving_distance_in_meters_from_api(row['coor'],x))

In [ ]:
driving_distances_df

## 3. Heatmap

Visualize which cars are demanded by which customers in which regions of Germany.

In [25]:
analysis_input_df = pd.read_csv('zip_code_analysis_input.csv',usecols=['sub_property_handover_zipcode','sub_property_handover_city','abs_net_purchase_price','brand_name','config_model_name','finn_car_id','deal_id','dim_fkey_pipelinestage','product_brand','purchasing_model','purchasing_model_line','product_fuel','helper_subscription_handover_date','product_body_type','delivery_compound_location'],dtype='str')
analysis_input_df

,sub_property_handover_zipcode,sub_property_handover_city,abs_net_purchase_price,brand_name,config_model_name,finn_car_id,deal_id,dim_fkey_pipelinestage,product_brand,purchasing_model,purchasing_model_line,product_fuel,helper_subscription_handover_date,product_body_type,delivery_compound_location
0,71397,Leutenbach,NaN,Fiat,500,dbancy4k,9711269437,1050363,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
1,90762,Fürth,NaN,Fiat,500,dbancy4k,2848683464,1319394,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
2,10405,Berlin,NaN,Fiat,500,sp2yog9g,2759857602,1319394,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
3,42109,WUPPERTAL,NaN,Fiat,500,sp2yog9g,7799851902,1050363,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
4,31812,Bad Pyrmont,NaN,Fiat,500,f8ljc1na,2241777119,1050363,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14757,81671,München,0,VW,Passat Variant,nsjx7u8v,9973803959,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14758,97789,Oberleichtersbach,0,VW,Passat Variant,ipe3lqjz,10683388270,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14759,45701,Herten,0,VW,Arteon Shooting Brake,h9esqm6m,11039441060,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14760,13587,Berlin,0,VW,Arteon Shooting Brake,nsi1icbe,11165345079,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
airtable_df = pd.read_csv('airtable_data.csv',dtype='str')
airtable_df

,finn_car_id,product_brand,purchasing_model,purchasing_model_line,product_fuel,helper_subscription_handover_date,product_body_type,delivery_compound_location
0,0a0jlxzj,Opel,Crossland,Edition,Benzin,NaN,SUV,compound_blg_saalanderdonau
1,0a4q0kww,Renault,Koleos,Intens,Benzin,2022-10-24,SUV,compound_rrg_eching
2,0a5aqavf,Jeep,Compass,Upland,Plug-In-Hybrid,NaN,SUV,compound_mosolf_kippenheim
3,0a5o8f6t,Audi,Q3,S line,Benzin (mild-hybrid),NaN,SUV,compound_akb_kitzingen
4,0a6jwfvb,Nissan,Qashqai,Acenta,Benzin (mild-hybrid),2022-06-02,SUV,compound_blg_d2c_atn2_neuss
...,...,...,...,...,...,...,...,...
42181,zzx60fa5,BMW,3er Touring,M Automobile,Diesel,2022-11-21,Kombi,compound_akb_kitzingen
42182,zzyasour,Jeep,Compass,Limited,Benzin,2022-06-28,SUV,compound_mosolf_kippenheim
42183,zzyg5ckg,Opel,Grandland,GS Line,Benzin,NaN,SUV,dealer_siebrecht_d2c_uslar
42184,zzz1n8rw,VW,Caravelle T6 1,Comfortline LR,Diesel,2023-02-09,Van,compound_akb_kitzingen


In [29]:
full_joined_df = pd.merge(analysis_input_df,airtable_df,how='inner',on='finn_car_id')
full_joined_df

,sub_property_handover_zipcode,sub_property_handover_city,abs_net_purchase_price,brand_name,config_model_name,finn_car_id,deal_id,dim_fkey_pipelinestage,product_brand_x,purchasing_model_x,...,helper_subscription_handover_date_x,product_body_type_x,delivery_compound_location_x,product_brand_y,purchasing_model_y,purchasing_model_line_y,product_fuel_y,helper_subscription_handover_date_y,product_body_type_y,delivery_compound_location_y
0,71397,Leutenbach,NaN,Fiat,500,dbancy4k,9711269437,1050363,Fiat,NaN,...,NaN,NaN,NaN,Fiat,500,Lounge,Benzin (mild-hybrid),2022-09-15,Kleinwagen,compound_mosolf_ketzin
1,90762,Fürth,NaN,Fiat,500,dbancy4k,2848683464,1319394,Fiat,NaN,...,NaN,NaN,NaN,Fiat,500,Lounge,Benzin (mild-hybrid),2022-09-15,Kleinwagen,compound_mosolf_ketzin
2,10405,Berlin,NaN,Fiat,500,sp2yog9g,2759857602,1319394,Fiat,NaN,...,NaN,NaN,NaN,Fiat,500,Lounge,Benzin (mild-hybrid),2022-02-22,Kleinwagen,compound_mosolf_ketzin
3,42109,WUPPERTAL,NaN,Fiat,500,sp2yog9g,7799851902,1050363,Fiat,NaN,...,NaN,NaN,NaN,Fiat,500,Lounge,Benzin (mild-hybrid),2022-02-22,Kleinwagen,compound_mosolf_ketzin
4,31812,Bad Pyrmont,NaN,Fiat,500,f8ljc1na,2241777119,1050363,Fiat,NaN,...,NaN,NaN,NaN,Fiat,500,Lounge,Benzin (mild-hybrid),2020-07-17,Kleinwagen,compound_mosolf_ketzin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,89275,Elchingen,0,VW,Passat Variant,bu0nians,10748691205,1050363,NaN,NaN,...,NaN,NaN,NaN,VW,Passat Variant,Business,Benzin,2022-12-22,Kombi,compound_arsaltmann_wolnzach
14757,81671,München,0,VW,Passat Variant,nsjx7u8v,9973803959,1050363,NaN,NaN,...,NaN,NaN,NaN,VW,Passat Variant,Business,Benzin,2022-09-26,Kombi,compound_arsaltmann_wolnzach
14758,97789,Oberleichtersbach,0,VW,Passat Variant,ipe3lqjz,10683388270,1050363,NaN,NaN,...,NaN,NaN,NaN,VW,Passat Variant,Business,Benzin,2022-11-11,Kombi,compound_arsaltmann_wolnzach
14759,45701,Herten,0,VW,Arteon Shooting Brake,h9esqm6m,11039441060,1050363,NaN,NaN,...,NaN,NaN,NaN,VW,Arteon Shooting Brake,R,Benzin,2022-12-15,Kombi,compound_akb_kitzingen
